# Notebook for generating plots of trained agents

This code will load the results for the specified experiments, generate plots, and save them in the `experiments_dir` directory

In [204]:
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import pandas as pd

In [205]:
params = {'legend.fontsize': 6,
                  "figure.autolayout": True,
                  'font.size': 8,
                  'pdf.fonttype':42,
                  'ps.fonttype':42}
plt.rcParams.update(params)

cm = 1 / 2.54  # for converting inches to cm
fig_size = (10.48 * cm, 6 * cm)  # these dimensions chosen to fit in latex column
    

* Set the directory in which all the experiments to be plotted together are stored.
* Set the list of subdirs (experiments) you want to include in the plots

In [206]:

!ls /media/elena/LaCie/aamas_2023/3_landmarks/classic_reward/2


ls: cannot access '/media/elena/LaCie/aamas_2023/3_landmarks/classic_reward/2': No such file or directory


In [207]:

experiments_dir = "/media/elena/LaCie/aamas_2023/6_landmarks/cooperative/2"
#experiments_dir = "/media/elena/LaCie/elias_expe/2_agents/cooperative/modified_reward/3_landmarks"
#list_of_experiments = ["centralized","independent","50align","naming_game_t30_a10"]
list_of_experiments =  [o for o in os.listdir(experiments_dir) if os.path.isdir(experiments_dir + "/" + o)]
print(list_of_experiments)

#list_of_experiments = ["centralized","independent"]

['centralized', 'naming_game_2_steps', 'naming_game_30msg_t30_a10', 'independent']


In [208]:
results = {}
for experiment in list_of_experiments:
    subdir = os.path.join(experiments_dir, experiment)
    eval_reward = pd.DataFrame()
    train_reward = pd.DataFrame()
    episode_len = pd.DataFrame()
    alignment = pd.DataFrame()
    train_x = []
    eval_x = []
    ctr=0
    for j, experiment_name in enumerate(os.listdir(subdir)):
        
        #print(subdir+"/"+experiment_name)
        if(os.path.isdir(subdir+"/"+experiment_name)):
          
            ctr+=1
            if(experiment_name[:5]!="p_est" and experiment_name[:5]!="0_5di"):
            
                if( (experiment_name[9]=="5" ) or experiment!="naming_game"):
                    print(experiment_name)

                    r = []
                    r_t = []
                    l = []
                    x_ = []
                    a = []
                    y_ = []
                    result_raw = open(os.path.join(subdir, experiment_name, "result.json"), "r")
                    #print(result_raw)

                    for result in result_raw:
                        dump = json.loads(result)
                        
                        #if(ctr>5):
                            #print(dump.keys())
                        a.append(dump["custom_metrics"].get("goal_alignment_mean"))
                        y_.append(dump["timesteps_total"])
                        r_t.append(dump["episode_reward_mean"])

                        metrics = dump.get("evaluation")
                        if metrics is not None:
                            custom = metrics.get("custom_metrics")
                            x_.append(dump["timesteps_total"])
                            r.append(metrics["episode_reward_mean"])
                            l.append(metrics["episode_len_mean"])

                    eval_reward = pd.concat(
                        [eval_reward, pd.DataFrame(r)], ignore_index=True, axis=1
                    )
                    episode_len = pd.concat(
                        [episode_len, pd.DataFrame(l)], ignore_index=True, axis=1
                    )
                    alignment = pd.concat([alignment, pd.DataFrame(a)], ignore_index=True, axis=1)
                    train_reward = pd.concat(
                        [train_reward, pd.DataFrame(r_t)], ignore_index=True, axis=1
                    )


                    if len(x_) > len(eval_x):
                        eval_x = x_
                    if len(y_) > len(train_x):
                        train_x = y_
    print(experiment)
    results[experiment] = {
        "eval_reward": eval_reward,
        "train_reward": train_reward,
        "episode_len": episode_len,
        "alignment": alignment,
        "train_x": train_x,
        "eval_x": eval_x,
    }

2022-10-05_14-27-44gdu_himi
2022-10-05_14-27-52ure2zpjr
2022-10-05_14-27-59vmaz9yqg
2022-10-05_14-27-440zxf_z9b
2022-10-05_14-27-48l1dt6vqn
centralized


IndexError: string index out of range

Training plots

In [ ]:
plt.figure(figsize=fig_size)

slice=1000

i = 1
n_exp = len(results)
for label, result in results.items():
    if "naming_game" in label:
        label_to_print = "naming_game"
    else:
        label_to_print = label

    
    plt.subplot(121)
    plt.plot(result["train_x"][:slice], result["train_reward"][:slice].mean(axis=1), label=label_to_print)
    plt.fill_between(
        result["train_x"][:slice],
        result["train_reward"][:slice].mean(axis=1) - result["train_reward"][:slice].std(axis=1),
        result["train_reward"][:slice].mean(axis=1) + result["train_reward"][:slice].std(axis=1),
        alpha=0.4,
    )
    if i == n_exp:
        plt.legend()
        #plt.grid()
        plt.ylabel("Training Reward, $R_{train}$")
        plt.xlabel("Timestep, \n $t_{train}$")

    plt.subplot(122)



    plt.plot(result["train_x"][:slice], result["alignment"][:slice].mean(axis=1), label=label_to_print)
    plt.fill_between(
        result["train_x"][:slice],
        result["alignment"][:slice].mean(axis=1) - result["alignment"][:slice].std(axis=1),
        result["alignment"][:slice].mean(axis=1) + result["alignment"][:slice].std(axis=1),
        alpha=0.4,
    )
    if i == n_exp:
        #plt.legend()
        #plt.grid()
        plt.ylabel("Alignment, $A$")
        plt.xlabel("Timestep,\n  $t_{train}$")

    i += 1

plt.tight_layout()
# Uncomment this line for saving the plot
plt.savefig(os.path.join(experiments_dir, "train_plot.png"), dpi=300, bbox_inches='tight', transparent=True)

Eval plots

In [ ]:
plt.figure(figsize=fig_size)
slice=100
i = 1
n_exp = len(results)
for label, result in results.items():
    if "naming_game" in label:
        label_to_print = "naming_game"
    else:
        label_to_print = label
    ax=plt.subplot(121)
    plt.plot(result["eval_x"][:slice], result["eval_reward"][:slice].mean(axis=1), label=label_to_print)
    plt.fill_between(
        result["eval_x"][:slice],
        result["eval_reward"][:slice].mean(axis=1) - result["eval_reward"][:slice].std(axis=1),
        result["eval_reward"][:slice].mean(axis=1) + result["eval_reward"][:slice].std(axis=1),
        alpha=0.4,
    )
    if i == n_exp:
        plt.legend()
        #plt.grid()
        plt.xlabel("Timestep, $t_{eval}$")
        plt.ylabel("Evaluation Reward, $R_{eval}$")

    plt.subplot(122)
    plt.plot(result["eval_x"][:slice], result["episode_len"][:slice].mean(axis=1), label=label_to_print)
    plt.fill_between(
        result["eval_x"][:slice],
        result["episode_len"][:slice].mean(axis=1) - result["episode_len"][:slice].std(axis=1),
        result["episode_len"][:slice].mean(axis=1) + result["episode_len"][:slice].std(axis=1),
        alpha=0.4,
    )
    if i == n_exp:
        #plt.legend()
        #plt.grid()
        plt.xlabel("Timestep, \n $t_{eval}$")
        plt.ylabel("Episode length, $L$")
    i += 1

plt.tight_layout()
# Uncomment this line for saving the plot
plt.savefig(os.path.join(experiments_dir, "evaluation_plot.png"), dpi=300, bbox_inches='tight', transparent=True)